<a href="https://colab.research.google.com/github/lehai-ml/KCL_AI_Workshops/blob/master/Machine_Learning_Workshop_3_Recurrent_Neural_Network_and_Kanye_Rap_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning Workshop 3: Recurrent Neural Network and Kanye Rap Generator and Sentiment Analysis
*Instructor:* Le Hai, (le.hai@kcl.ac.uk)
[KCL Artificial Intelligence Society](https://www.facebook.com/kclaisoc/?__xts__[0]=68.ARBS16FGqFmhg52L83VtU55PY45QGKlU8Jrej-r7ZHALKkj9E4dzZMvGfDvg8oHnazPAlEJQ80M7--6XTHbh6hyvOoplBMVsWLITGjI5PEc8xf-ap7n00ASBE92fNKalpTFcK2OlA0hVimQEDNWEmrQkWmIQ_UiAqxDR9CQuD8YMZNBUIVOpnatRb-AKTJy5xAwKYyBa7kaBUHVsgqoCqx6BUHqgg6_0wQUoRKvWAak1mE5Vdq4uGSPpTcFtaUJz4z1iHQUaUYUjpYcMDtmuu6LFRiMnXkZaQa4YEguXNsCSBmw2tAeMX3gOXMPke-mSHO6FxwFE7Mymu0zkHZu67Mo&eid=ARB2iiXncCNlQsYa-y6uPQ7o0e_PN2CjhwNmnyTVAgy9emKkNf0k27tbD1PgM2vK_zkbN5DoiGECvZ82)

**Workshop was modified from:** https://www.tensorflow.org/tutorials/text/word_embeddings and
https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/text/text_classification_rnn.ipynb#scrollTo=Bq6xDmf2SAs- and 
https://colab.research.google.com/github/fastai/course-v3/blob/master/nbs/dl1/lesson3-imdb.ipynb#scrollTo=Hu3BON7_4Z3f
and 
https://towardsdatascience.com/machine-learning-text-classification-language-modelling-using-fast-ai-b1b334f2872d


Here, we will mainly utilize the **fast.ai** library, since the syntax is intuitive and readable

If you want to create a model from scratch, see here:

https://www.kaggle.com/utkarshnag/text-generation-using-shakespeare-plays


![Unrolled RNN](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png)

![LSTM Cell unit](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png)

# **Kanye Rap lyrics generator**



In [ ]:
from tensorflow import keras # import necessary libraries of tensorflow, and keras API
from tensorflow.keras import layers
from fastai.text import * # we will also use fastai library to deal with text
import numpy as np # necessary for math operation
import pandas as pd #necessary for dataframe operation

In [ ]:
# Dataset
import urllib.request #this is for opening url
# kanye lyrics: https://pastebin.com/raw/9S5u08EU
target_url = "https://pastebin.com/raw/9S5u08EU" # go to pastebin, paste in your rap lyrics, submit that and then click "raw", or swap it out for one of the URL's above
dirty_rap_source = urllib.request.urlopen(target_url).read()
rap_source = [x.split(b'\r')[0].decode('utf-8') for x in dirty_rap_source.splitlines()]#I open the url source, and then read line by line
                                                                                        # and then save it to rap_source as list
while "" in rap_source: #remove unecessary spaces
  rap_source.remove("")
while " " in rap_source:
  rap_source.remove(" ")
print ("Your dataset's first 10 rap lines: ")
rap_source[:10]

Your dataset's first 10 rap lines: 


['Let the suicide doors up',
 'I threw suicides on the tour bus',
 'I threw suicides on the private jet',
 "You know what that mean, I'm fly to death",
 "I step in Def Jam buildin' like I'm the shit",
 "Tell 'em give me fifty million or I'ma quit",
 "Most rappers' taste level ain't at my waist level",
 "Turn up the bass 'til it's up-in-yo-face level",
 "Don't do no press but I get the most press kit",
 'Plus, yo, my bitch make your bitch look like Precious']

## Data processing

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_set,test_set=train_test_split(rap_source,test_size=0.1) #divide into train and test set

In [ ]:
df_trn=pd.DataFrame(train_set)# save train and test set into dataframes so we can better manipulate them
df_test=pd.DataFrame(test_set)

In [ ]:
df_trn

,0
0,"I'm trippin', I'm caught up in the moment, right?"
1,"Tell the conductor, ""Just drive so far."""
2,You ain't accustomed to going through customs
3,Niggas tryin to figure out since Kan came
4,Where they drivin' spaceships instead of cars
...,...
5566,"Out of state though, man that hurt, yo"
5567,"In my past, you on the other side of the glass"
5568,"Only tell you real shit, that's the tea, no sip"
5569,"I'm not loving you, way I wanted to"


In [ ]:
#Now we are going to call the fast.ai library to prepare our text for training (https://docs.fast.ai/text.data.html#NLP-datasets)
data_lm = TextLMDataBunch.from_df('./',train_df=df_trn, valid_df=df_test,text_cols=0)
#in nutshell, we will:
#1)tokenize and numerize our dataset
#2)divide our dataset into batches for training process

### **Tokenization** (behind the scene)


The easiest way to process our text is by spliting the string on spaces, but fast.ai do more than that by:
- take care of punctuation
- deals with contractions, i.e. isn't or don't.
- clean parts of the text, e.g. HTML code.

To see what the tokenizer had done behind the scenes, let's have a look at a few texts in a single batch.

In [ ]:
data_lm.show_batch(1)

idx,text
0,"xxunk nigga xxbos xxmaj just looking at your history xxbos "" xxmaj baby girl , he 's a loner xxbos xxmaj but i 'm fuckin ' , fuckin ' , fuckin ' like i 'm tryna make four more xxbos xxmaj black dick all in your spouse again xxbos xxmaj we lost a four leaf xxunk xxbos xxmaj started a little xxunk just to get some traffic xxbos xxmaj put"


Here, all sentences in a single batch have been concatenated into a single sentence. Not only words have been divided by at the space, but certain tokens have been used to denotes begining of the sentence, capitalization of a word, and contractions are tokenized as: did and n't etc.

**xxbos**: Begining of a sentence

**xxfld**: Represent separate parts of a document like title, summary etc., each one will get a separate field and so they will get numbered (e.g. xxfld 1, xxfld 2).

**xxup**: If there's something in all caps, it gets lower cased and a token called xxup will get added to it. Words that are fully capitalized, such as “I AM SHOUTING”, are tokenized as “xxup i xxup am xxup shouting“

**xxunk**: token used instead of an uncommon word.

**xxmaj**: token indicates that there is capitalization of the word. “The” will be tokenized as “xxmaj the“.

**xxrep**: token indicates repeated word, if you have 29 ! in a row, (i.e. xxrep 29 !).



### **Numerization** (behind the scene)


With our tokens, we convert them into integers, such that our "new dictionary" will keep the ones that appear at least twice, and the rest will have the token "UNK"

Our vocabulary (from int to token) is stored in 'vocab' attribute, in a dictionary called 'itos'.

Let's look at it

In [ ]:
print(data_lm.vocab.itos[:30]) #here is our token (vocab) list

['xxunk', 'xxpad', 'xxbos', 'xxeos', 'xxfld', 'xxmaj', 'xxup', 'xxrep', 'xxwrep', ',', 'i', 'the', 'you', 'a', 'and', 'to', "n't", 'that', 'my', "'", 'it', "'s", 'in', 'me', "'m", 'like', 'we', 'do', 'on', '?']


In [ ]:
data_lm.train_ds[4][0] # confirm again that each line has been separated into tokens

Text xxbos xxmaj where they drivin ' xxunk instead of cars

In [ ]:
data_lm.train_ds[4][0].data #and underlying data is actually represented by numbers, confirm that hater- no. 1287 is repeated twice.

array([   2,    5,  108,   31,  779,   19,    0, 1048,   36,  489])

## **Transfer learning**

Similar to previously examined Convolutional neural network, you do not want to build a language model from scratch. Here, we utilize a RNN model, that has been previously learnt on wikipedia text called AWD_LSTM (https://arxiv.org/pdf/1708.02182.pdf). 

The biggest advantage of using a pre-trained model is faster training time for our model, since the pre-trained model will have better understanding of the language (here, English) than from scratch.

In [ ]:
learner = language_model_learner(data_lm, AWD_LSTM) #we pass our dataset to the AWD_LSTM

In [ ]:
# learner.lr_find()
# learner.recorder.plot() #if you want to play around with the learning rate.

In [ ]:
learner.fit_one_cycle(1, 2e-2,moms=(0.8,0.7)) # here we fit our dataset 1 cycle, with learning rate of 2e-2.

epoch,train_loss,valid_loss,accuracy,time
0,5.174777,4.095222,0.238281,00:02


In [ ]:
learner.unfreeze() # we then unfreeze the rest of the language model, and train the model on the whole dataset.
learner.fit_one_cycle(10, 2e-2,moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.261035,3.828128,0.294643,00:02
1,4.076240,3.733333,0.298549,00:02
2,3.885966,3.651770,0.319420,00:02
3,3.740411,3.700088,0.313393,00:02
4,3.527023,3.685466,0.315179,00:02
5,3.287577,3.772322,0.316518,00:02
6,3.063703,3.895697,0.312165,00:02
7,2.776875,3.992608,0.311719,00:02
8,2.496473,4.083580,0.312277,00:02
9,2.271247,4.090904,0.311942,00:03


## Final Result

In [ ]:
TEXT = "Kanye"
N_WORDS = 1000
prediction=learner.predict(TEXT, N_WORDS, temperature=0.8) #temperature determines how randomized you want your words to be.
prediction.split('xxbos')

["Kanye , got that , that 's a whole new arena ",
 ' Got a couple cases , still look good ',
 " And i 'm , way too exclusive ",
 ' He said homie you got your own dick , you got ta have two kids ',
 ' She said you left your kids and they just like you ',
 ' Is that bitch really Benz ? ',
 ' She said , " You gon na kill me , then you do n\'t believe me " ',
 ' So i could make it from the student loans to a Benz ',
 ' Folks all up in our business ',
 ' Rose gold Jesus piece with the brown ice ',
 ' The crowd is open , i see the glass , i know the same thing ',
 " Make 'em say bout we getting through , nigga ",
 " So i could make it to the church ' fore you meet me ",
 ' You still got ta go and get the CD on me ',
 ' Step into the beat , throw a movie on ',
 ' i can never be too big for my team ',
 " Try and make it to the party ' fore the guest list close ",
 " If you do n't know by now i 'm talkin about Chi - Town ",
 ' And just a few things i see with my eyes closed ',
 " We used to do 

# another example: **Shakespearean play generator** 



In [ ]:
# Dataset
import urllib.request #this is for opening url
# kanye lyrics: https://pastebin.com/raw/9S5u08EU

target_url = "https://pastebin.com/raw/EyjV8ffZ" # go to pastebin, paste in your rap lyrics, submit that and then click "raw", or swap it out for one of the URL's above
dirty_rap_source = urllib.request.urlopen(target_url).read()
rap_source = [x.split(b'\r')[0].decode('utf-8') for x in dirty_rap_source.splitlines()]#I open the url source, and then read line by line
#                                                                                         # and then save it to rap_source as list
rap_source=[i.replace('"','') for i in rap_source]

from sklearn.model_selection import train_test_split
train_set,test_set=train_test_split(rap_source,test_size=0.1) #divide into train and test set
df_trn=pd.DataFrame(train_set)
df_test=pd.DataFrame(test_set)
data_lm = TextLMDataBunch.from_df('./',train_df=df_trn, valid_df=df_test,text_cols=0)
learner_Shakespear = language_model_learner(data_lm, AWD_LSTM,drop_mult=0.5)
learner_Shakespear.fit_one_cycle(1, 2e-2,moms=(0.8,0.7)) # here we fit our dataset 1 cycle, with learning rate of 2e-2.
learner_Shakespear.unfreeze() # we then unfreeze the rest of the language model, and train the model on the whole dataset.
learner_Shakespear.fit_one_cycle(10, 2e-2,moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.770309,4.029067,0.279911,00:02


epoch,train_loss,valid_loss,accuracy,time
0,3.956928,3.823079,0.303943,00:03
1,3.815207,3.937426,0.274479,00:03
2,3.735601,3.753656,0.304539,00:03
3,3.634139,5.096787,0.122842,00:03
4,3.650626,3.848439,0.297098,00:03
5,3.405970,3.811265,0.305060,00:03
6,3.061718,3.923314,0.306771,00:03
7,2.671323,4.067795,0.305060,00:03
8,2.315563,4.164559,0.301860,00:03
9,2.055634,4.200223,0.302307,00:03


In [ ]:
TEXT = "To be or not to be"
N_WORDS = 500
prediction=learner_Shakespear.predict(TEXT, N_WORDS, temperature=0.8) #temperature determines how randomized you want your words to be.
prediction.split('xxbos')

['To be or not to be contented by thee : ',
 ' i am no loathsome and write too good . ',
 ' And , to crave the end of war , being never ',
 " Than i from France and England 's fair - bought Welsh , ",
 ' And so he comes , but yet the foe is merciless , ',
 ' Great King Henry , York , Buckingham , York , ',
 ' My lord , a prize ! a prize ! here should i take . ',
 ' The king his son , Oxford , shall resign his crown . ',
 ' Speak French , and , hear me speak , speak , ',
 ' But Ned , that Bolingbroke ',
 ' wherein will she pay the answer ? ',
 ' As are of better person than myself . ',
 " And for his safety there she 'll shun pursuit . ",
 ' Which in the time of death he gave : ',
 " Whom , knowing Dame Eleanor 's aspiring humour , ",
 ' And so farewell , and yet , methinks , no more , ',
 ' For Suffolk , he that died i do , ',
 ' As is the difference and day of this , ',
 " That Clifford 's pride may be borne . ",
 ' that i do assure my counsel . ',
 ' It is impossible that i should di

# **Sentiment Analysis: IMDB review classification**

Taken from: https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/text_classification_with_hub.ipynb#scrollTo=2ew7HTbPpCJH

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

!pip install tensorflow-hub
!pip install tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

TensorFlow 2.x selected.
Version:  2.1.0-rc1
Eager mode:  True
Hub version:  0.7.0
GPU is available


In [ ]:
# Split the training set into 60% and 40%, so we'll end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

In [ ]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)) # call up the examples
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [ ]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [ ]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1" # use the pre-trained embeddings
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])# doesn't matter how long the sentences are, the embedding input will always have the same dimension.

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

<tf.Tensor: shape=(1, 20), dtype=float32, numpy=
array([[ 0.8771677 , -4.3859434 ,  4.5861564 , -1.8083508 , -2.7040975 ,
        -2.61265   , -2.56528   ,  0.85366595,  4.640281  , -1.0381072 ,
        -1.4069302 ,  0.25684452,  0.08177447,  0.88873774, -3.1487586 ,
         3.3401423 ,  2.351665  , -1.933979  , -2.0873525 , -0.9392492 ]],
      dtype=float32)>

Open the [Embedding Projector](http://projector.tensorflow.org/)

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================] - 5s 168ms/step - loss: 2.0095 - accuracy: 0.5009 - val_loss: 0.7945 - val_accuracy: 0.5354
Epoch 2/20
30/30 [==============================] - 4s 134ms/step - loss: 0.6958 - accuracy: 0.5910 - val_loss: 0.6575 - val_accuracy: 0.6160
Epoch 3/20
30/30 [==============================] - 4s 136ms/step - loss: 0.6460 - accuracy: 0.6341 - val_loss: 0.6354 - val_accuracy: 0.6370
Epoch 4/20
30/30 [==============================] - 4s 133ms/step - loss: 0.6265 - accuracy: 0.6566 - val_loss: 0.6205 - val_accuracy: 0.6526
Epoch 5/20
30/30 [==============================] - 4s 139ms/step - loss: 0.6087 - accuracy: 0.6747 - val_loss: 0.6062 - val_accuracy: 0.6694
Epoch 6/20
30/30 [==============================] - 4s 139ms/step - loss: 0.5910 - accuracy: 0.6915 - val_loss: 0.5912 - val_accuracy: 0.6836
Epoch 7/20
30/30 [==============================] - 4s 137ms/step - loss: 0.5725 - accuracy: 0.7082 - val_loss: 0.5744 - val_accuracy: 0.7000
Epoch 

In [ ]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

loss: 0.351
accuracy: 0.847


In [ ]:
string=tf.constant('I love this movie')
model.predict(tf.reshape(string,(1,)))

array([[0.84114534]], dtype=float32)